In [2]:
#!/bin/python3
import os
import joblib
import pandas as pd
from flask import Flask, jsonify, request
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson_openscale import APIClient
from ibm_watson_openscale.supporting_classes.enums import DataSetTypes, TargetTypes
from ibm_watson_openscale.supporting_classes.payload_record import PayloadRecord

from dotenv import load_dotenv
load_dotenv()



True

In [3]:

ibmcloudapikey = os.getenv("ibmcloudapikey")
SERVICE_INSTANCE_ID = os.getenv("SERVICE_INSTANCE_ID")
subscription_id = os.getenv("subscription_id")

# Load the machine learning model
model = joblib.load("auto-ai-pipeline.pickle")


In [4]:
app = Flask(__name__)

In [66]:
def vik():
    output="This REST API call predicts the demand response of a customer. \nUse POST /v2/predict with your payload!\nUse POST /v2/feedback_logging with your payload and response to load data into Watson OpenScales feedback_logging table!\nUse POST /v2/payload_logging with your payload to load data and response into Watson OpenScales payload_logging table!"
    return str(output)


In [8]:
@app.route("/v2/greet", methods=["GET"])
def status():
    global model
    if model is None:
        return "Flask Code: Model was not loaded."
    else:
        return "Demand Response Model is loaded."


In [ ]:
def predict(payload):
    global model
    payload = payload['input_data'][0]
    df = pd.DataFrame(payload['values'], columns=payload['fields'])
    
    probs = model.predict_proba(df.values).tolist()
    preds = model.predict(df.values).tolist()
    res = [{'prediction': preds[i], 'probability': probs[i]} for i in range(len(preds))]
    output = {
        'predictions': [{
            "fields": ['prediction', 'probability'],
            "values": [[res[i]['prediction'], res[i]['probability']] for i in range(len(res))]
        }]
    }

    
@app.route("/v2/predict", methods=["POST"])
def predict_demand_response():

    payload = request.get_json()
    output=predict(payload)
    return jsonify(output)
    



In [53]:
@app.route("/v2/authenticate", methods=["POST"])
def openscale_authentication():
    try:
        service_credentials = {
        "apikey": ibmcloudapikey,
        "url": "https://api.aiopenscale.cloud.ibm.com"
        }

        authenticator = IAMAuthenticator(
                apikey=service_credentials["apikey"],
                url="https://iam.cloud.ibm.com/identity/token"
            )

        wos_client = APIClient(authenticator=authenticator, service_instance_id=SERVICE_INSTANCE_ID, service_url=service_credentials["url"])
        return wos_client
    except:
        return "Watson OpenScale Client Error"


In [39]:
@app.route("/v2/feedback_logging", methods=["POST"])
def feedback_logging():
    payload = request.get_json()
    predictions=predict(payload)
    feedback_log_req={}
    feedback_log_req['fields']=test_input['input_data'][0]['fields']+["DEMAND_RESPONSE"]+["_original_prediction","_original_probability","_debiased_prediction","_debiased_probability"]
    feedback_log_req['values']=[]
    for x in range(len(predictions['predictions'][0]['values'])):
         feedback_log_req['values'].append(test_input['input_data'][0]['values'][x]+[int(predictions['predictions'][0]['values'][x][0])]+predictions['predictions'][0]['values'][x]+predictions['predictions'][0]['values'][x])

        
    
    try:
        wos_client = openscale_authentication()
        feedback_dataset_id = wos_client.data_sets.list(type=DataSetTypes.FEEDBACK, target_target_id=subscription_id, target_target_type=TargetTypes.SUBSCRIPTION).result.data_sets[0].metadata.id


        wos_client.data_sets.store_records(
                data_set_id=feedback_dataset_id,
                request_body=[feedback_log_req],
                background_mode=False
        )
        return "Feedback Logging Successfull"

    except:
        return "Feedback Logging Failed, Please check your watson openscale instance and verify the inputs"




In [60]:
@app.route("/v2/payload_logging", methods=["POST"])
def payload_logging():
    payload = request.get_json()
    payload_scoring_request=payload['input_data'][0]
    payload_scoring_request['meta']={'fields':['referrer_gender'], 'values':[[i[8]] for i in payload['input_data'][0]['values']]}
    payload_scoring_response = predict(payload)
    
    try:
        wos_client = openscale_authentication()
        payload_logging_data_set_id = wos_client.data_sets.list(type=DataSetTypes.PAYLOAD_LOGGING, target_target_id=subscription_id, target_target_type=TargetTypes.SUBSCRIPTION).result.data_sets[0].metadata.id
        
        wos_client.data_sets.store_records(data_set_id=payload_logging_data_set_id, request_body=[PayloadRecord(request=payload_scoring_request, response=payload_scoring_response, response_time=460)])
        return "Payload Logging Successful"
    except:
        return "Payload Logging Failed, Please check your watson openscale instance and verify the inputs"
        



In [ ]:
test_input={'input_data': [{'fields': ['CUSTOMER_ID','CSTFNM','CSTLNM','CSTPH1','CSTML','AGE','CITY','MARITAL_STATUS','GENDER','EDUCATION','EMPLOYMENT','TENURE','SEGMENT','HOME_SIZE','ENERGY_USAGE_PER_MONTH','ENERGY_EFFICIENCY','IS_REGISTERED_FOR_ALERTS','OWNS_HOME','COMPLAINTS','EST_INCOME','CLTV','HAS_THERMOSTAT','HAS_HOME_AUTOMATION','PHOTOVOLTAIC_ZONING','WIND_ZONING','SMART_METER_COMMENTS','IS_CAR_OWNER','HAS_EV','HAS_PHOTOVOLTAIC','HAS_WIND','EBILL','IN_WARRANTY','STD_YRLY_USAGE','MISSED_PAYMENT','YEARLY_USAGE_PREDICTED'],
   'values': [[1,'Leonor','Cummerata','781-757-3537','Leonor.Cummerata@nola.biz',44,'Mountain View','S','male','Associate degree','Employed full-time',13,'Green Advocate',1980,5080,0.39,True,True,False,47870,352.0,True,False,True,False,'Negative',True,True,True,False,False,True,56049,False,59972.4],
             [106,'Alberta','Hackett','520-430-9350','Alberta.Hackett@albina.info',52,'Santa Clara','M','female','High school graduate','Employed full-time',14,'Budget Payment Plan Members',2120,3340,0.635,True,True,False,65619,141.0,True,True,False,True,'Positive',True,True,False,False,True,True,28395,False,28395.0]]}]}

predictions=predict(test_input)


In [75]:
test_input={'input_data': [{'fields': ['CUSTOMER_ID','CSTFNM','CSTLNM','CSTPH1','CSTML','AGE','CITY','MARITAL_STATUS','GENDER','EDUCATION','EMPLOYMENT','TENURE','SEGMENT','HOME_SIZE','ENERGY_USAGE_PER_MONTH','ENERGY_EFFICIENCY','IS_REGISTERED_FOR_ALERTS','OWNS_HOME','COMPLAINTS','EST_INCOME','CLTV','HAS_THERMOSTAT','HAS_HOME_AUTOMATION','PHOTOVOLTAIC_ZONING','WIND_ZONING','SMART_METER_COMMENTS','IS_CAR_OWNER','HAS_EV','HAS_PHOTOVOLTAIC','HAS_WIND','EBILL','IN_WARRANTY','STD_YRLY_USAGE','MISSED_PAYMENT','YEARLY_USAGE_PREDICTED'],
   'values': [[1,'Leonor','Cummerata','781-757-3537','Leonor.Cummerata@nola.biz',44,'Mountain View','S','male','Associate degree','Employed full-time',13,'Green Advocate',1980,5080,0.39,True,True,False,47870,352.0,True,False,True,False,'Negative',True,True,True,False,False,True,56049,False,59972.4],
             [106,'Alberta','Hackett','520-430-9350','Alberta.Hackett@albina.info',52,'Santa Clara','M','female','High school graduate','Employed full-time',14,'Budget Payment Plan Members',2120,3340,0.635,True,True,False,65619,141.0,True,True,False,True,'Positive',True,True,False,False,True,True,28395,False,28395.0]]}],
           'service_credentials':{'ibmcloudapikey':'KDuYn_OWhzQ4O5zqoo9mHEWv-ouFMiATJcUGJAUJFOZo','SERVICE_INSTANCE_ID':'b78d9fcd-74b7-4b17-a6bf-1983299bafc5','subscription_id':'ce86bff6-2994-45a7-b06c-f12eb0bbb9a9'}
           }


In [73]:
test_input['service_credentials']['ibmcloudapikey']

'KDuYn_OWhzQ4O5zqoo9mHEWv-ouFMiATJcUGJAUJFOZo'

In [69]:
SERVICE_INSTANCE_ID

'b78d9fcd-74b7-4b17-a6bf-1983299bafc5'

In [81]:

def openscale_authentication_logging(credentials):
    try:
        service_credentials = {
        "apikey": credentials['ibmcloudapikey'],
        "url": "https://api.aiopenscale.cloud.ibm.com"
        }

        authenticator = IAMAuthenticator(
                apikey=service_credentials["apikey"],
                url="https://iam.cloud.ibm.com/identity/token"
            )

        wos_client = APIClient(authenticator=authenticator, service_instance_id=credentials['SERVICE_INSTANCE_ID'], service_url=service_credentials["url"])
        return wos_client
    except:
        return "Watson OpenScale Client Error"




@app.route("/v2/predict_and_logging", methods=["POST"])
def predict_and_logging():
    payload = request.get_json()
    payload_scoring_request=payload['input_data'][0]
    payload_scoring_request['meta']={'fields':['referrer_gender'], 'values':[[i[8]] for i in payload['input_data'][0]['values']]}
    payload_scoring_response = predict(payload)
    predictions=payload_scoring_response
    feedback_log_req={}
    feedback_log_req['fields']=payload['input_data'][0]['fields']+["DEMAND_RESPONSE"]+["_original_prediction","_original_probability","_debiased_prediction","_debiased_probability"]
    feedback_log_req['values']=[]
    for x in range(len(predictions['predictions'][0]['values'])):
         feedback_log_req['values'].append(payload['input_data'][0]['values'][x]+[int(predictions['predictions'][0]['values'][x][0])]+predictions['predictions'][0]['values'][x]+predictions['predictions'][0]['values'][x])

    wos_client = openscale_authentication_logging(payload['service_credentials'])
    payload_logging_data_set_id = wos_client.data_sets.list(type=DataSetTypes.PAYLOAD_LOGGING, target_target_id=payload['service_credentials']['subscription_id'], target_target_type=TargetTypes.SUBSCRIPTION).result.data_sets[0].metadata.id

    wos_client.data_sets.store_records(data_set_id=payload_logging_data_set_id, request_body=[PayloadRecord(request=payload_scoring_request, response=payload_scoring_response, response_time=460)])
    print("payload logging successful")
    feedback_dataset_id = wos_client.data_sets.list(type=DataSetTypes.FEEDBACK, target_target_id=payload['service_credentials']['subscription_id'], target_target_type=TargetTypes.SUBSCRIPTION).result.data_sets[0].metadata.id


    wos_client.data_sets.store_records(
            data_set_id=feedback_dataset_id,
            request_body=[feedback_log_req],
            background_mode=False
    )
    
    print("feedback logging successful")
    output={"response":"Payload Logging and Feedback logging Successful","model_prediction":payload_scoring_response}


    return jsonify(output)


In [82]:
predict_and_logging(test_input)

payload logging successful



 Waiting for end of storing records with request id: da2c7576-5504-49f4-9e61-63b90a52171e 




active

---------------------------------------
 Successfully finished storing records 
---------------------------------------


feedback logging successful


{'response': 'Payload Logging and Feedback logging Successful',
 'model_prediction': {'predictions': [{'fields': ['prediction', 'probability'],
    'values': [[0.0, [0.6210089111178061, 0.37899108888219385]],
     [1.0, [0.31457225323657423, 0.6854277467634258]]]}]}}